In [ ]:
import os

# Thư mục chứa ảnh
image_dir = "../data/images_test"
image_paths = []

# Duyệt qua các thư mục con và thêm đường dẫn các tệp hình ảnh vào danh sách
for root, dirs, files in os.walk(image_dir):
    for file in files:
        image_path = os.path.join(root, file)
        image_paths.append(image_path)

# In số lượng tệp hình ảnh
print("Số lượng tệp hình ảnh: ", len(image_paths))

Số lượng tệp hình ảnh:  118

In [ ]:
import cv2
from deepface import DeepFace
import time
from tqdm import tqdm

# Các thông số đầu vào
detectors = ['opencv', 'ssd', 'mtcnn', 'retinaface']
confidence = 0.9
save_path = "../data/images_test_detection"

# Khởi tạo biến đếm khuôn mặt và thời gian
face_counts = {'opencv': 0, 'ssd': 0, 'mtcnn': 0, 'retinaface': 0}
time_total = {'opencv': 0, 'ssd': 0, 'mtcnn': 0, 'retinaface': 0}

for detector in detectors:
    # tạo thư mục chứa ảnh gương mặt
    path = os.path.join(save_path, detector)
    if not os.path.exists(path):
        os.makedirs(path)

    count = 0
    _time = 0
    for image_path in tqdm(image_paths):
        try:
            # trích xuất khuôn mặt từ ảnh
            start_time = time.time()
            results = DeepFace.extract_faces(img_path=image_path, detector_backend=detector, enforce_detection=False,
                                             align=False)
            end_time = time.time()

            result = results[0]
            if result['confidence'] > confidence:
                count += 1
                # lưu khuôn mặt
                rgb_image = cv2.cvtColor(result['face'], cv2.COLOR_BGR2RGB)
                img_uint8 = (rgb_image * 255).astype('uint8')
                if not cv2.imwrite(os.path.join(path, "face_" + str(count) + ".jpg"), img_uint8):
                    raise Exception("Could not write image")
            _time += (end_time - start_time)

        except Exception as e:
            print("Error occurred: ", e)
            continue
    print("Complete:", detector, ', count:', count, ', time:', round(_time, 3))
    # Lưu lại kết quả
    face_counts[detector] = count
    time_total[detector] = round(_time, 3)

for detector in detectors:
    print(detector + "=>", "face:", face_counts[detector], "time:", time_total[detector])


opencv=> face: 92 time: 17.787
ssd=> face: 116 time: 5.551
mtcnn=> face: 118 time: 158.74
retinaface=> face: 118 time: 396.326